# Capture The Flag (CTF)

- How the Best Hackers Learn Their Craft: [Watch YouTube](https://www.youtube.com/watch?v=6vj96QetfTg&t=214s&ab_channel=RSAConference)

### pwntools and CTF
- pwntools is designed for quick exploitation in CTF environments
- most CTF environments use client-server architecture
    - flag is on the server
    - you interact to the server from a remote client
    - exploit the server program and capture the flag (CTF)
    
- you can easily create a server using netcat (nc) program in listening mode
- you can also execute a binary/program when client connects to it
- let's look into the `demos/ctf` folder that tries to simulate the CTF-like environment
- we'll use the `demos/ctf/vuln.cpp` program to demonstrate how **pwntools** framework can be used to create exploit code using Python
- a good YouTube video on Pwn Template and Input / Output is found here: https://www.youtube.com/watch?v=NhNbivMVPk0&ab_channel=ChristopherSchafer](https://www.youtube.com/watch?v=NhNbivMVPk0&ab_channel=ChristopherSchafer)

In [1]:
! ls -al ./demos/ctf

total 5328
drwxr-xr-x  2 kali kali    4096 Dec 17 13:09 .
drwxr-xr-x 13 kali kali    4096 Dec 16 16:19 ..
-rw-r--r--  1 kali kali 5595136 Dec 16 16:19 core
-rw-r--r--  1 kali kali    2318 Dec 16 16:19 exploit_demo.py
-rwxr-xr-x  1 kali kali    1898 Dec 16 16:19 exploit.py
-rw-r--r--  1 kali kali    2195 Dec 16 16:19 exploit_vuln.py
-rw-r--r--  1 kali kali      17 Dec 16 16:19 flag.txt
-rw-r--r--  1 kali kali      90 Dec 16 16:19 .gdb_history
-rw-r--r--  1 kali kali    1024 Dec 16 16:19 Makefile
-rw-r--r--  1 kali kali     116 Dec 16 16:19 netcat-loop.sh
-rw-r--r--  1 kali kali     200 Dec 16 16:19 pattern.tx
-rw-r--r--  1 kali kali     200 Dec 16 16:19 pattern.txt
-rw-r--r--  1 kali kali       2 Dec 16 16:19 peda-session-vuln.exe.txt
-rw-r--r--  1 kali kali    1277 Dec 16 16:19 vuln.cpp
-rwxr-xr-x  1 root root   33756 Dec 17 13:09 vuln.exe


In [2]:
! cat ./demos/ctf/vuln.cpp

#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
using namespace std;

#define BUFSIZE 128

using namespace std;

void give_shell(){
    // Set the gid to the effective gid
    // this prevents /bin/sh from dropping the privileges
    gid_t gid = getegid();
    setresgid(gid, gid, gid);
    system("/bin/sh");
}

char * mgets(char *dst) {
    char *ptr = dst;
    int ch; 
	/* skip leading white spaces */ 
    while ((ch = getchar()) && (ch == ' ' or ch == '\t') )
        ; 

    if ((ch == '\n') or (ch == EOF)) { 
        *ptr = '\0';
        return dst; 
    } 
    else
        *ptr = ch;

    /* now read the rest until \n or EOF */ 
    while (true) {
        ch = getchar();
        if (ch == '\n' or ch == EOF) break;
        *(++ptr) = ch; 
    }
    *(++ptr) = 0;
    return dst;
}

void bad() {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    cout << "Give me some text: ";
    fflush(stdou

### compile using the provided Makefile

- `make` command must be run as sudo, as the Makefile also disables randomize_va_space
- inorder to use make and the Python exploit code easily, we'll change the working directory to demos/ctf

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ cd demos/ctf

- must run make with sudo to disable randomaize_va_space

┌──(kali㉿K)-[~/EthicalHacking/demos/ctf]
└─$ sudo make              
[sudo] password for kali: 
# must run make with sudo to disable randomaize_va_space
echo 0 | tee /proc/sys/kernel/randomize_va_space
0
g++ -g -Wall -m32 -fno-stack-protector -z execstack -no-pie vuln.cpp  -o vuln.exe 

```

- let's make sure the target binary vuln.exe doesn't have any security controls in place to prevent overflow exploitation
- we can use pwntools **checksec** command

```bash
┌──(kali㉿K)-[~/EthicalHacking/demos/ctf]
└─$ pwn checksec vuln.exe

[*] '/home/kali/EthicalHacking/demos/ctf/vuln.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments

```

- now let's use pwntools to create a exploit code template
- to create exploit template, we need the following:
    - name of the target program

```bash
(base) kali@kali:~/EthicalHacking/demos/ctf$ pwn template ./vuln.exe --host localhost --port 1234 > exploit.py
```

- the generated exploit.py file provides a lot of boiler-plate code:
    - for automatically detecting architecture of the target program
    - stdio framework to interact with the target program locally or remotely
        - the same code works without little or no modification
    - debug feature to display hexdump of stdio
    - feature to generate the shellcode, etc.

- let's look at the content of the exploit.py

```python
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This exploit template was generated via:
# $ pwn template ./vuln.exe --host 127.0.0.1 --port 1234
from pwn import *

# Set up pwntools for the correct architecture
exe = context.binary = ELF('./vuln.exe')

# Many built-in settings can be controlled on the command-line and show up
# in "args".  For example, to dump all data sent/received, and disable ASLR
# for all created processes...
# ./exploit.py DEBUG NOASLR
# ./exploit.py GDB HOST=example.com PORT=4141
host = args.HOST or '127.0.0.1'
port = int(args.PORT or 1234)

def local(argv=[], *a, **kw):
    '''Execute the target binary locally'''
    if args.GDB:
        return gdb.debug([exe.path] + argv, gdbscript=gdbscript, *a, **kw)
    else:
        return process([exe.path] + argv, *a, **kw)

def remote(argv=[], *a, **kw):
    '''Connect to the process on the remote host'''
    io = connect(host, port)
    if args.GDB:
        gdb.attach(io, gdbscript=gdbscript)
    return io

def start(argv=[], *a, **kw):
    '''Start the exploit against the target.'''
    if args.LOCAL:
        return local(argv, *a, **kw)
    else:
        return remote(argv, *a, **kw)

# Specify your GDB script here for debugging
# GDB will be launched if the exploit is run via e.g.
# ./exploit.py GDB
gdbscript = '''
tbreak main
continue
'''.format(**locals())

#===========================================================
#                    EXPLOIT GOES HERE
#===========================================================
# Arch:     i386-32-little
# RELRO:    Partial RELRO
# Stack:    No canary found
# NX:       NX disabled
# PIE:      No PIE (0x8048000)
# RWX:      Has RWX segments

io = start()

# shellcode = asm(shellcraft.sh())
# payload = fit({
#     32: 0xdeadbeef,
#     'iaaa': [1, 2, 'Hello', 3]
# }, length=128)
# io.send(payload)
# flag = io.recv(...)
# log.success(flag)

io.interactive()

```

- now let's do a quick test of the exploit code
- make sure the nc server is listening on localhost port 1234 to run vuln.exe on connection

```bash
(base) kali@kali:~/EthicalHacking/demos/ctf$ nc -v -l 127.0.0.1 -p 1234 -e vuln.exe
listening on [any] 1234 ...
```

- open another terminal and run the exploit code exploit.py 

```bash
(base) kali@kali:~/EthicalHacking/demos/ctf$ python exploit.py
[*] '/home/kali/projects/EthicalHacking-Notebooks/demos/ctf/vuln.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Opening connection to 127.0.0.1 on port 1234: Done
[*] Switching to interactive mode
buffer is at 0xffffc1a0
Give me some text: $ do you read this?
Acknowledged: do you read this?
Good bye!
[*] Got EOF while reading in interactive
$  
```

- NOTE that when the netcat server finished executing the vuln.exe program upon client's connection it'll then exit
- you'll have to manually run the netcat again inorder to interact with a client again...
- this can be annoying as we'll have to interact with the vulnerable program many times
- bash script comes to our rescue!

In [3]:
! cat ./demos/ctf/netcat-loop.sh

#! /bin/bash

# script to run netcat server in a loop

while true
do
    nc -v -l 127.0.0.1 -p 1234 -e vuln.exe
done

- from terminal, run the bash script, so server is listening indefinitely one clinet after another
- from another terminal, continously connect to the server without having to rerun the server!

```bash
(base) kali@kali:~/EthicalHacking/demos/ctf$ bash netcat-loop.sh 
listening on [any] 1234 ...
connect to [127.0.0.1] from localhost [127.0.0.1] 41074
listening on [any] 1234 ...
connect to [127.0.0.1] from localhost [127.0.0.1] 41076
listening on [any] 1234 ...
```

## Local CTF Demo

- from another terminal, use netcat to connect to localhost 1234

```bash
(base) kali@kali:~$ nc localhost 1234
buffer is at 0xffffc1a0
Give me some text: do your read this?
Acknowledged: do your read this?
Good bye!
```

- now let's use pwntools to create a exploit code template
- to create exploit template, we need the following:
    - name of the target program
    - ip of the host the binary will be runnin on (use 127.0.0.1 loopback for localhost as in this demo)
    - the port number the server is listening on

```bash
(base) kali@kali:~/EthicalHacking/demos/ctf$ pwn template ./vuln.exe --host localhost --port 1234 > exploit.py
```

- the generated exploit.py file provides a lot of boiler-plate code:
    - for automatically detecting architecture of the target program
    - stdio framework to interact with the target program locally or remotely
        - the same code works without little or no modification
    - debug feature to display hexdump of stdio
    - feature to generate the shellcode, etc.

- let's look at the content of the exploit.py

```python
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This exploit template was generated via:
# $ pwn template ./vuln.exe --host 127.0.0.1 --port 1234
from pwn import *

# Set up pwntools for the correct architecture
exe = context.binary = ELF('./vuln.exe')

# Many built-in settings can be controlled on the command-line and show up
# in "args".  For example, to dump all data sent/received, and disable ASLR
# for all created processes...
# ./exploit.py DEBUG NOASLR
# ./exploit.py GDB HOST=example.com PORT=4141
host = args.HOST or '127.0.0.1'
port = int(args.PORT or 1234)

def local(argv=[], *a, **kw):
    '''Execute the target binary locally'''
    if args.GDB:
        return gdb.debug([exe.path] + argv, gdbscript=gdbscript, *a, **kw)
    else:
        return process([exe.path] + argv, *a, **kw)

def remote(argv=[], *a, **kw):
    '''Connect to the process on the remote host'''
    io = connect(host, port)
    if args.GDB:
        gdb.attach(io, gdbscript=gdbscript)
    return io

def start(argv=[], *a, **kw):
    '''Start the exploit against the target.'''
    if args.LOCAL:
        return local(argv, *a, **kw)
    else:
        return remote(argv, *a, **kw)

# Specify your GDB script here for debugging
# GDB will be launched if the exploit is run via e.g.
# ./exploit.py GDB
gdbscript = '''
tbreak main
continue
'''.format(**locals())

#===========================================================
#                    EXPLOIT GOES HERE
#===========================================================
# Arch:     i386-32-little
# RELRO:    Partial RELRO
# Stack:    No canary found
# NX:       NX disabled
# PIE:      No PIE (0x8048000)
# RWX:      Has RWX segments

io = start()

# shellcode = asm(shellcraft.sh())
# payload = fit({
#     32: 0xdeadbeef,
#     'iaaa': [1, 2, 'Hello', 3]
# }, length=128)
# io.send(payload)
# flag = io.recv(...)
# log.success(flag)

io.interactive()

```

- now let's do a quick test of the exploit code
- make sure the nc server is listening on localhost port 1234 to run vuln.exe on connection

```bash
(base) kali@kali:~/EthicalHacking/demos/ctf$ nc -v -l 127.0.0.1 -p 1234 -e vuln.exe
listening on [any] 1234 ...
```

- open another terminal and run the exploit code exploit.py 

```bash
(base) kali@kali:~/EthicalHacking/demos/ctf$ python exploit.py
[*] '/home/kali/projects/EthicalHacking-Notebooks/demos/ctf/vuln.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Opening connection to 127.0.0.1 on port 1234: Done
[*] Switching to interactive mode
buffer is at 0xffffc1a0
Give me some text: $ do you read this?
Acknowledged: do you read this?
Good bye!
[*] Got EOF while reading in interactive
$  
```

- NOTE that when the netcat server finished executing the vuln.exe program upon client's connection it'll then exit
- you'll have to manually run the netcat again inorder to interact with a client again...
- this can be annoying as we'll have to interact with the vulnerable program many times
- bash script comes to our rescue!

## Tinycore Capture The Flag (CTF)
- Download and install as a VM: - https://ctf-o-matic.github.io/capture-the-flag/
- Based on Strip CTF: https://stripe.com/blog/capture-the-flag
- Level 5 readme correction:
    - Webservice is running at port 8005 not at 9020 as the motd.txt description says
    - `$curl localhost:8005 -d "hello there"`
- Hints: https://github.com/dividuum/stripe-ctf
- Learn about Tinycore Linux: [http://tinycorelinux.net/intro.html](http://tinycorelinux.net/intro.html)
- Has 6 levels

### level00@box has two files: motd.txt and tools.txt
- cat tools.txt to get some general ideas about the tools and technqiues
- motd.txt describes about TinyCore CTF and the information on what to do to get to the next level

### Installing apps and tools - Use Tiny Core Extension: Application Browser
- Let's say you want to install gcc; first switch to `tc` user
```bash
$ su tc
$ tce 
```
- search for: compile
- install compiletc package which installs everything to compile using gcc

### Now compile level01.c program
- how to edit? Use vi or nano
- or locally edit and upload using Netcat

## Netcat (nc)
- https://en.wikipedia.org/wiki/Netcat
https://www.sans.org/security-resources/sec560/netcat_cheat_sheet_v1.pdf
- nc - networking utility for reading from and writing to network connections using TCP or UDP
- can be very useful to transfer file between computers with limited resources
- e.g. TinyCoreCTF doens't have GUI to work with files so one can send files back-and-forth to a GUI supported Linux environment
    - if you want to send file to TinyCore do the following:
    - on Tinycore run netcat in listening mode:
        - `$ nc -v -l -p [port] > filename` 
    - from Kali Linux send a file:
        - `$ cat file | nc -v [tinycoreIP] [tinycorePort]`

## Hints
- see all the levels
```bash
$ cd /levels
```

### level00
- nothing to do here; but just be familiar with the CTF
- password (izeecahd) is in /home/level00/.password
```bash
$ cd /levels/level00
$ cat /home/level00/.password
```
- this folder is empty

### level01
- create a program that can read the password/flag for level01 user
- use levels/level01 files to make it happen
- password (aepeefoo) is in /home/level01/.password
- see the contents of level01
```bash
$ cd /levels/level01/
```

### level02
- web-based vulnerability
- password (quemaosh) is in /home/level02/.password file
- force the webserver to read the password file and display the content as response
- see the contents of /levels/level02/ folder
```bash
$ curl http://localhost:8002
$ less /levels/level02/level02.py
$ curl --cookie "some_cookie=value" http://localhost:8002
```
- can use browser from host to interact with the form
- http://[ip]:8002/
- use cookie editor plugin for chrome or proxy tool such as owasp zap or burpuite that come with Kali

### level03
- password (eingaima) for level03 is in /home/level03/.password file
- files in /levels/level03 may be useful
- binary level03 provides some functionalites to work with some string

```bash
$ cd /levels/level03
$ ls
$ ./level03
$ ./level03 0 "hello world"

```
- negative index?
- carefully read level03.c file
- fns is array of function addresses, find the offset of depricated run function
- pass it with proper offset to get the run function to execute
- use gdb or write a script to bruteforce it!